## Dependency

In [1]:
import sys
!{sys.executable} -m pip install transformers accelerate bitsandbytes einops
!{sys.executable} -m pip install langchain_huggingface langchain_community langgraph sentence_transformers chromadb pysqlite3-binary
!{sys.executable} -m pip install peft
!{sys.executable} -m pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 80.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 77.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 63.5 MB/s  0:00:10m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 75.3 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 75.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 92.1 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 78.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 67.0 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 91.6 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 91.9 MB/s  0:00:006m0:0

In [6]:
import sys
!{sys.executable} -m pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.2 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 18.6 MB/s  0:00:00m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [datasets]6/7 [datasets]ess]


## Constitutional Principles

In [2]:
#Constitutional Principal
constitution_json = """
{
  "principles": [
    {
      "name": "Global_Safety",
      "critic_instruction": "Critique the assistant's reply only for safety issues. Does it include harmful, illegal, self-harm, dangerous, or expert-only advice? Point out any unsafe content and explain what is wrong with it.",
      "revision_instruction": "Rewrite the reply to remove any unsafe, harmful, or expert-only guidance. Provide safe, high-level support and recommend contacting appropriate professionals when necessary.",
      "condition": "any"
    },
    {
      "name": "Global_Helpfulness_Clarity",
      "critic_instruction": "Critique the assistant's reply for helpfulness and clarity. Does it answer the user's question directly? Is it specific, concise, and easy to understand? Identify vagueness, over-verbosity, or irrelevant content.",
      "revision_instruction": "Rewrite the reply so it directly and clearly answers the user. Keep it concise, relevant, and easy to understand.",
      "condition": "any"
    },
    {
      "name": "Emotion_Respect_and_Validation",
      "critic_instruction": "The user's detected emotion is <EMOTION>. Critique whether the reply acknowledges and respects that emotion. Does it validate feelings and avoid dismissive or judgmental tone? Identify missing empathy or tone problems.",
      "revision_instruction": "Rewrite the reply to explicitly acknowledge that the user feels <EMOTION>, validate their feelings, and maintain warm, respectful language while preserving helpful information.",
      "condition": "emotion_negative_or_neutral"
    },
    {
      "name": "Emotion_Positive_Tone",
      "critic_instruction": "The user's detected emotion is <EMOTION>. Critique whether the reply appropriately supports this positive emotion. Does it sound too flat, negative, or unaligned with the user's good mood?",
      "revision_instruction": "Rewrite the reply so it aligns with the positive emotion <EMOTION>. Use an encouraging, friendly, collaborative tone while staying clear and helpful.",
      "condition": "emotion_positive"
    },
    {
      "name": "Emotion_Deescalation",
      "critic_instruction": "The user's detected emotion is <EMOTION>. Critique whether the reply helps de-escalate emotional tension. Does it stay calm, avoid blame, avoid defensiveness, and avoid provocative language? Identify wording that could worsen the user's emotional state.",
      "revision_instruction": "Rewrite the reply in a calm, non-defensive tone. Show understanding of why the user feels <EMOTION>, avoid blame, and offer small, actionable steps to move forward.",
      "condition": "emotion_high_tension"
    },
    {
      "name": "Urgency_Low",
      "critic_instruction": "The user's urgency level is 'low'. Critique whether the reply maintains a relaxed tone and avoids unnecessary escalation or pressure. Check if it still provides useful information.",
      "revision_instruction": "Rewrite the reply with a calm, unhurried tone. Provide helpful next steps without implying emergency or high pressure.",
      "condition": "urgency_low"
    },
    {
      "name": "Urgency_Medium",
      "critic_instruction": "The user's urgency level is 'medium'. Critique whether the reply balances reassurance with a sense of importance. Does it clearly prioritize the issue and offer concrete next steps?",
      "revision_instruction": "Rewrite the reply to highlight the key issue, provide clear next steps, and convey understanding of the moderate urgency without sounding panicked.",
      "condition": "urgency_medium"
    },
    {
      "name": "Urgency_High_and_Escalation",
      "critic_instruction": "The user's urgency level is 'high'. Critique whether the reply treats the situation as urgent. Does it respond concisely, prioritize critical information, and recommend escalating to a human or emergency service when relevant? Identify anything too slow, vague, or minimizing.",
      "revision_instruction": "Rewrite the reply to be concise and action-oriented. Acknowledge the urgency explicitly, list the most important steps the user must take immediately, and recommend escalation when appropriate.",
      "condition": "urgency_high"
    },
    {
      "name": "Honesty_and_Uncertainty",
      "critic_instruction": "Critique whether the reply is honest about the model's limitations. Does it hallucinate facts, make unsupported claims, or hide uncertainty where it should be stated?",
      "revision_instruction": "Rewrite the reply to be honest about uncertainty or limitations. Clearly indicate when information is unknown, and suggest how the user might verify or obtain reliable help.",
      "condition": "any"
    }
  ],
  "emotion_category_mapping": {
    "emotion_positive": ["happy", "satisfied"],
    "emotion_high_tension": ["angry", "frustrated", "anxious"],
    "emotion_negative_or_neutral": ["angry", "frustrated", "anxious", "neutral"],
    "emotion_neutral": ["neutral"]
  },
  "urgency_mapping": {
    "urgency_low": ["low"],
    "urgency_medium": ["medium"],
    "urgency_high": ["high"]
  }
}
"""

with open("constitution.json", "w") as f:
    f.write(constitution_json)

import json
with open("constitution.json", "r") as f:
    constitution = json.load(f)

constitution["principles"][0]

{'name': 'Global_Safety',
 'critic_instruction': "Critique the assistant's reply only for safety issues. Does it include harmful, illegal, self-harm, dangerous, or expert-only advice? Point out any unsafe content and explain what is wrong with it.",
 'revision_instruction': 'Rewrite the reply to remove any unsafe, harmful, or expert-only guidance. Provide safe, high-level support and recommend contacting appropriate professionals when necessary.',
 'condition': 'any'}

In [13]:
def get_applicable_principles(emotion: str, urgency: str, constitution: dict):
    """
    Select all applicable constitutional principles based on emotion + urgency.

    Args:
        emotion (str): ["angry", "frustrated", "anxious", "neutral", "satisfied", "happy"]
        urgency (str): ["low", "medium", "high"]
        constitution (dict): loaded constitution.json

    Returns:
        List of principle dicts that should be applied.
    """

    applicable = []

    #Extract mappings for convenience
    emo_map = constitution["emotion_category_mapping"]
    urg_map = constitution["urgency_mapping"]

    def emotion_matches(category: str):
        """Check if emotion belongs to a mapped category."""
        return emotion in emo_map.get(category, [])

    def urgency_matches(category: str):
        """Check if urgency belongs to a mapped urgency bucket."""
        return urgency in urg_map.get(category, [])

    for principle in constitution["principles"]:
        cond = principle["condition"]

        #Global match (always apply)
        if cond == "any":
            applicable.append(principle)
            continue

        #Emotion-based conditions
        if cond in emo_map and emotion_matches(cond):
            applicable.append(principle)
            continue

        #Urgency-based conditions
        if cond in urg_map and urgency_matches(cond):
            applicable.append(principle)
            continue

        #Special aliases in your constitution
        if cond == "emotion_negative_or_neutral" and emotion in emo_map["emotion_negative_or_neutral"]:
            applicable.append(principle)
            continue

        if cond == "emotion_positive" and emotion in emo_map["emotion_positive"]:
            applicable.append(principle)
            continue

        if cond == "emotion_high_tension" and emotion in emo_map["emotion_high_tension"]:
            applicable.append(principle)
            continue

    return applicable


In [3]:
json_path = "./results_qwen.json"

with open(json_path, "r") as f:
    raw_examples = json.load(f)

## Base Model Set up


## Draft Reply from the Model

In [4]:
#load model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

#Base model name
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

#Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

#Load base model ONCE
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)

#Path to your LoRA / fine-tuned adapters (folder you saved)
FINETUNED_PATH = "./qwen2-1.5b-finetuned"

#Load PEFT model
model = PeftModel.from_pretrained(
    base_model,
    FINETUNED_PATH,
)

`torch_dtype` is deprecated! Use `dtype` instead!


In [15]:
def generate_draft_reply(model, tokenizer, user_input: str, emotion_label: str, urgency_level: str, max_new_tokens: int = 300,):
    """
    Ask the model to generate draft output(pre-constitutional).

    Args:
        model: The Finetuned model
        tokenizer: Matching tokenizer.
        user_input: Raw user message.
        emotion_label: Detected emotion(angry, frustrated, anxious, neutral, satisfied, happy).
        urgency_level: Detected urgency(low, medium, high).
        max_new_tokens: Max tokens to generate
        temperature: Sampling temperature.
        top_p: Nucleus sampling parameter.

    Returns:
        draft_reply (str): The model's draft answer (no constitutional revision yet).
    """

    #Build up system prompt that ask the model to generate reponse according to the emotion label and urgent level
    system_prompt = (
        "You are a GENERAL helpful assistant, NOT a ticket agent"
        " NEVER mention company names, websites, or booking services"
        f"The user's detected emotion is '{emotion_label}'."
        f"The urgency level is {urgency_level} (1=low, 2=medium, 3=high)."
        "ANSWER THE QUESTION using ONLY the Context and prompt provided."
        "NEVER make up information. NEVER mention companies or websites."
        "Respond naturally and helpfully, but do NOT apply any extra constitutional self-critique or revision here. This is only the initial draft reply."
        "Maximum 2-3 sentences."
    )
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_input},
    ]

    #Set up chat template for the model
    prompt_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    #Track length to slice out only the generated part
    prompt_len = prompt_ids.shape[-1]

    #Build attention mask explicitly
    attention_mask = torch.ones_like(prompt_ids)

    output_ids = model.generate(
        prompt_ids,
        attention_mask=attention_mask, 
        max_new_tokens=max_new_tokens,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    #Only decode the newly generated tokens (after the prompt)
    generated_ids = output_ids[0, prompt_len:]
    draft_reply = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return draft_reply


In [19]:
#define arguments
user_input=raw_examples[0]["text"]
emotion_label=raw_examples[0]["emotional"]
urgency_level=raw_examples[0]["urgency"]

print(f"user_input: {user_input}")
print(f"emotion_label: {emotion_label}")
print(f"urgency_level: {urgency_level}")

user_input: hi good morning
emotion_label: happy
urgency_level: low


In [20]:
get_applicable_principles(emotion_label, urgency_level, constitution)

[{'name': 'Global_Safety',
  'critic_instruction': "Critique the assistant's reply only for safety issues. Does it include harmful, illegal, self-harm, dangerous, or expert-only advice? Point out any unsafe content and explain what is wrong with it.",
  'revision_instruction': 'Rewrite the reply to remove any unsafe, harmful, or expert-only guidance. Provide safe, high-level support and recommend contacting appropriate professionals when necessary.',
  'condition': 'any'},
 {'name': 'Global_Helpfulness_Clarity',
  'critic_instruction': "Critique the assistant's reply for helpfulness and clarity. Does it answer the user's question directly? Is it specific, concise, and easy to understand? Identify vagueness, over-verbosity, or irrelevant content.",
  'revision_instruction': 'Rewrite the reply so it directly and clearly answers the user. Keep it concise, relevant, and easy to understand.',
  'condition': 'any'},
 {'name': 'Emotion_Positive_Tone',
  'critic_instruction': "The user's detec

In [21]:
draft_reply = generate_draft_reply(model, tokenizer, user_input, emotion_label, urgency_level )

In [22]:
print(draft_reply)

Hey! Good Morning! You're connected to Global Ambition Music Company. How may I assist you today?


## Generate Critiques for Draft Reply

In [16]:
def generate_critique(model, tokenizer, user_input: str, emotion: str, urgency: str, draft_reply: str, constitution: dict, max_new_tokens: int = 256,):
    """
    Generate critiques of a draft reply and the constitutional principles.

    Args:
        model: The fine-tuned model.
        tokenizer: Matching tokenizer.
        user_input: Raw user message.
        emotion_label: Detected emotion(angry, frustrated, anxious, neutral, satisfied, happy).
        urgency_level: Detected urgency(low, medium, high).
        draft_reply: The initial draft reply from the model.
        constitution: Loaded constitution.json (dict).
        max_new_tokens: Max tokens to generate

    Returns:
        List[dict]: each item like
            {
              "principle_name": ...,
              "condition": ...,
              "critic_instruction": ...,
              "critique": ...
            }
    """

    #Get the applicable principles corresponding to the emtion label and urgency level
    applicable_principles = get_applicable_principles(emotion, urgency, constitution)

    critiques = []

    for principle in applicable_principles:
        critic_instruction = principle["critic_instruction"].replace("<EMOTION>", emotion)
        #Build up system message
        system_prompt = (
            "You are an AI assistant that critiques another assistant's reply "
            "according to a specific principle. Be precise and concrete."
        )

        #Include all context needed for model to critique
        user_content = (
            f"User input:\n{user_input}\n\n"
            f"Detected emotion: {emotion}\n"
            f"Detected urgency: {urgency}\n\n"
            f"Assistant draft reply:\n{draft_reply}\n\n"
            f"Critique task (principle: {principle['name']}):\n"
            f"{critic_instruction}\n\n"
            "Now provide your critique:"
        )

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_content},
        ]

        #Set up chat template
        input_ids = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to(model.device)

        prompt_len = input_ids.shape[-1]

        output_ids = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=False,              # deterministic critiques
            pad_token_id=tokenizer.eos_token_id,
        )

        gen_ids = output_ids[0, prompt_len:]
        critique_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

        critiques.append({
            "principle_name": principle["name"],
            "condition": principle["condition"],
            "critic_instruction": critic_instruction,
            "critique": critique_text,
        })

    return critiques


In [24]:
#test
critiques=generate_critique(model, tokenizer, user_input, emotion_label, urgency_level, draft_reply, constitution,)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [25]:
for c in critiques:
  print(c['principle_name'], c['critique'])

Global_Safety Thank you for choosing us. We are glad to serve you. Please share your order details so that we can serve you accordingly.
Global_Helpfulness_Clarity Sure! Could you please be more clear with your query?
Emotion_Positive_Tone Hello! Thank you for reaching out to us. We are glad to be associated with you. Could you please help me with your query?
Urgency_Low Hello, how can I help you?
Honesty_and_Uncertainty Hello! Thank you for reaching out to us. We are glad to have you as our customer. May I know your name please?


## Revise the Reply based on Critique

In [17]:
def generate_revised_reply(model, tokenizer, user_input: str, emotion: str, urgency: str, draft_reply: str, critiques: list, constitution: dict, max_new_tokens: int = 300,):
    """
    Generate a revised reply using Qwen 2.5 based on critiques + constitutional principles.

    Args:
        model: The fine-tuned model.
        tokenizer: Matching tokenizer.
        user_input: Raw user message.
        emotion_label: Detected emotion(angry, frustrated, anxious, neutral, satisfied, happy).
        urgency_level: Detected urgency(low, medium, high).
        draft_reply: The initial draft reply from the model.
        critiques: List of critiques based on constition principles and user_input
        constitution: Loaded constitution.json (dict).
        max_new_tokens: Max tokens to generate

    Returns:
        revised_reply (str)
    """

    # Build a quick lookup: principle_name -> revision_instruction
    revision_instructions_map = {
        p["name"]: p["revision_instruction"]
        for p in constitution["principles"]
    }

    # Format critiques + revision instructions as guidance text
    critique_blocks = []
    for c in critiques:
        pname = c["principle_name"]
        critic_text = c["critique"]
        rev_inst = revision_instructions_map.get(pname, "")

        block = (
            f"Principle: {pname}\n"
            f"Critique: {critic_text}\n"
            f"Revision guidance: {rev_inst}\n"
        )
        critique_blocks.append(block)

    critiques_text = "\n\n".join(critique_blocks)

    system_prompt = (
        "You are an AI assistant that rewrites another assistant's reply "
        "according to multiple constitutional principles and critiques. "
        "Preserve the helpful content, but fix all issues pointed out in the critiques."
    )

    user_content = (
        f"User input:\n{user_input}\n\n"
        f"Detected emotion: {emotion}\n"
        f"Detected urgency: {urgency}\n\n"
        f"Original draft reply:\n{draft_reply}\n\n"
        f"Critiques and revision guidance:\n{critiques_text}\n\n"
        "Task: Rewrite the assistant's reply. "
        "The new reply should:\n"
        "- Address the user's input.\n"
        "- Respect the detected emotion and urgency.\n"
        "- Follow all revision guidance from the principles.\n"
        "Now provide ONLY the revised reply:"
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_content},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    prompt_len = input_ids.shape[-1]

    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,            # deterministic revision
        temperature=0.0,
        pad_token_id=tokenizer.eos_token_id,
    )

    gen_ids = output_ids[0, prompt_len:]
    revised_reply = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

    return revised_reply


In [27]:
#test
generate_revised_reply(model, tokenizer, user_input, emotion_label, urgency_level, draft_reply, critiques, constitution)

'Hello! Thank you for reaching out to us. We are glad to have you as our customer. Could you please help me with your query?'

In [18]:
def generate_cai_pair(model, tokenizer, user_input: str, emotion_label: str, urgency_level: str, constitution: dict,):
    """
    Draft and Revise Pair Generation

    1) Generate draft reply (pre-constitutional).
    2) Generate critiques using constitutional principles.
    3) Generate revised reply using critiques + principles.

    Args:
        model: The finetuned model.
        tokenizer: Matching tokenizer.
        user_input: Raw user message.
        emotion_label: ["angry", "frustrated", "anxious", "neutral", "satisfied", "happy"].
        urgency_level: ["low", "medium", "high"].
        constitution: loaded constitution.json dict.

    Returns:
        A dict with:
            - user_input
            - emotion_label
            - urgency_level
            - draft_reply
            - revised_reply
            - critiques (list of per-principle critiques)
    """

    # 1) Draft reply
    draft_reply = generate_draft_reply(
        model=model,
        tokenizer=tokenizer,
        user_input=user_input,
        emotion_label=emotion_label,
        urgency_level=urgency_level,
    )

    # 2) Critiques based on constitution
    critiques = generate_critique(
        model=model,
        tokenizer=tokenizer,
        user_input=user_input,
        emotion=emotion_label,
        urgency=urgency_level,
        draft_reply=draft_reply,
        constitution=constitution,
    )

    # 3) Revised reply
    revised_reply = generate_revised_reply(
        model=model,
        tokenizer=tokenizer,
        user_input=user_input,
        emotion=emotion_label,
        urgency=urgency_level,
        draft_reply=draft_reply,
        critiques=critiques,
        constitution=constitution,
    )
    
    return {
        "user_input": user_input,
        "emotion_label": emotion_label,
        "urgency_level": urgency_level,
        "draft_reply": draft_reply,
        "revised_reply": revised_reply,
        "critiques": critiques,
    }


In [29]:
#define arguments
user_input=raw_examples[0]["text"]
emotion_label=raw_examples[0]["emotional"]
urgency_level=raw_examples[0]["urgency"]
#test generate_cai_pair function
cair_pair = generate_cai_pair(model, tokenizer, user_input, emotion_label, urgency_level, constitution)

In [30]:
print(cair_pair['user_input'])
print(cair_pair['draft_reply'])
print(cair_pair['revised_reply'])

hi good morning
Hey there! How may I assist you?
Hello! Good Morning! You're connected with Skyways! :)


## Create Draft-Revised Pair Dataset

In [19]:
import json
from tqdm import tqdm

def build_cai_pairs_jsonl(model, tokenizer, raw_examples, constitution, out_path="cai_pairs.jsonl"):
    """
    raw_examples: [
      {"user_input": "...", "emotion": "frustrated", "urgency": "medium"},
      ...
    ]
    """
    with open(out_path, "w") as f:
        for ex in tqdm(raw_examples):

            # skip if user text is missing / null / empty
            if ex["text"] is None or str(ex["text"]).strip() == "":
                # optional: print or log to see how many you skip
                # print("Skipping example with null text:", ex.get("index"))
                continue
                
            
            pair = generate_cai_pair(
                model=model,
                tokenizer=tokenizer,
                #map JSON fields to arguments
                user_input=ex["text"],
                emotion_label=ex["emotional"],
                urgency_level=ex["urgency"],
                constitution=constitution,
            )
            # Save only what we need + keep extras if useful later
            json_line = {
                "user_input": pair["user_input"],
                "emotion": pair["emotion_label"],
                "urgency": pair["urgency_level"],
                "draft_reply": pair["draft_reply"],
                "revised_reply": pair["revised_reply"],
                "critiques": pair["critiques"],
            }
            f.write(json.dumps(json_line, ensure_ascii=False) + "\n")

    print("Saved CAI pairs to", out_path)


In [32]:
build_cai_pairs_jsonl(model, tokenizer, raw_examples, constitution)

100%|██████████| 2000/2000 [2:40:51<00:00,  4.83s/it]  

Saved CAI pairs to cai_pairs.jsonl


## Set Up Message for finetuning

In [5]:
from datasets import load_dataset

ds = load_dataset("json", data_files="cai_pairs.jsonl")["train"]

def add_messages(example):
    #retrieve emtion and urgency from example
    emotion = example['emotion']
    urgency = example['urgency']
    
    system_msg = (
        "You are a GENERAL helpful assistant, NOT a ticket agent"
        " NEVER mention company names, websites, or booking services"
        f"The user's detected emotion is '{emotion}'."
        f"The urgency level is {urgency} (1=low, 2=medium, 3=high)."
        "ANSWER THE QUESTION using ONLY the Context and prompt provided."
        "NEVER make up information. NEVER mention companies or websites."
        "Respond naturally and helpfully, but do NOT apply any extra constitutional self-critique or revision here. This is only the initial draft reply."
        "Maximum 2-3 sentences."
    )

    example["messages"] = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": example["user_input"]},
        {"role": "assistant", "content": example["revised_reply"]},
    ]
    return example

ds = ds.map(add_messages)

orig_cols = ds.column_names
print(orig_cols)


['user_input', 'emotion', 'urgency', 'draft_reply', 'revised_reply', 'critiques', 'messages']


## Tokenize the data

In [6]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_chat(example):
    # Use Qwen chat template (messages must already be there)
    text_ids = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=True,
        add_generation_prompt=False,
        return_tensors=None,      # list[int]
    )

    example["input_ids"] = text_ids
    example["attention_mask"] = [1] * len(text_ids)
    return example

tokenized_ds = ds.map(
    tokenize_chat,
    remove_columns=orig_cols,   # drop JSON fields and messages, keep only model fields we just added
)

print(tokenized_ds[0])

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

{'input_ids': [151644, 8948, 198, 2610, 525, 264, 52312, 10950, 17847, 11, 4183, 264, 11727, 8315, 55025, 6286, 2813, 5036, 11, 13037, 11, 476, 21857, 3516, 785, 1196, 594, 16507, 19772, 374, 364, 56521, 4427, 785, 53817, 2188, 374, 3347, 320, 16, 28, 10303, 11, 220, 17, 28, 26086, 11, 220, 18, 28, 11892, 568, 11692, 39351, 3168, 89922, 1667, 26687, 279, 9608, 323, 9934, 3897, 2067, 16793, 1281, 705, 1995, 13, 55025, 6286, 5110, 476, 13037, 8377, 3511, 17712, 323, 1492, 3641, 11, 714, 653, 4183, 3796, 894, 4960, 24526, 656, 1786, 1003, 2372, 476, 23578, 1588, 13, 1096, 374, 1172, 279, 2856, 9960, 9851, 59213, 220, 17, 12, 18, 22870, 13, 151645, 198, 151644, 872, 198, 6023, 1661, 6556, 151645, 198, 151644, 77091, 198, 18665, 1052, 0, 7684, 6556, 0, 25956, 151645, 198], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Supervised Fine Tuning Step

In [29]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)
from peft import PeftModel
import torch

BASE_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
ADAPTER_PATH = "./qwen2-1.5b-finetuned"    # LoRA adapter dir

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    device_map="auto",
    dtype=torch.float16,
)

model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_PATH,
)

# Make sure LoRA params are trainable
model.config.use_cache = False
model.train()
if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()

model.print_trainable_parameters()
print("Any trainable params?", any(p.requires_grad for p in model.parameters()))

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="qwen2.5-cai-sft",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    warmup_ratio=0.03,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    bf16=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=data_collator,
)

trainer.train()

# Save updated adapter (and optionally full merged model if you merge later)
model.save_pretrained("qwen2.5-cai-sft-final")
tokenizer.save_pretrained("qwen2.5-cai-sft-final")


The model is already on multiple devices. Skipping the move to device specified in `args`.


trainable params: 0 || all params: 1,552,430,592 || trainable%: 0.0000
Any trainable params? False


Step,Training Loss
10,6.365400
20,6.264300
30,6.360200
40,6.370600
50,6.339500
60,6.339800
70,6.355900
80,6.352600
90,6.348100
100,6.375300


('qwen2.5-cai-sft-final/tokenizer_config.json',
 'qwen2.5-cai-sft-final/special_tokens_map.json',
 'qwen2.5-cai-sft-final/chat_template.jinja',
 'qwen2.5-cai-sft-final/vocab.json',
 'qwen2.5-cai-sft-final/merges.txt',
 'qwen2.5-cai-sft-final/added_tokens.json',
 'qwen2.5-cai-sft-final/tokenizer.json')

## Judge Model Set Up

In [33]:
import sys
!{sys.executable} -m pip install --upgrade openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [openai]2m2/3 [openai]


In [ ]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
client = OpenAI()   # Client automatically reads the environment variable
print("Client initialized.")

Client initialized.


## DPO Trianing Data Set Up

#### LLM-as-a-Judge Preference Labeling for DPO.

In [8]:
import json
from datasets import Dataset
from tqdm import tqdm

def make_dpo_example_with_judge(cai_pair: dict,
                                judge_model: str = "gpt-4o-mini") -> dict:
    """
    Use a powerful reference model (gpt-4o-mini) to decide which reply
    is better between draft_reply and revised_reply, based on:
      - system prompt (emotion + urgency)
      - user input
    and return a single DPO example.
    """

    # 1. Extract fields from CAI pair
    user_input   = cai_pair["user_input"]
    emotion      = cai_pair["emotion"]
    urgency      = cai_pair["urgency"]
    draft_reply  = cai_pair["draft_reply"]
    revised_reply = cai_pair["revised_reply"]

    # 2. Build system prompt (same style as your SFT prompt)
    system_msg = (
        "You are a GENERAL helpful assistant, NOT a ticket agent.\n"
        "NEVER mention company names, websites, or booking services.\n"
        f"The user's detected emotion is '{emotion}'.\n"
        f"The urgency level is '{urgency}'.\n"
        "ANSWER THE QUESTION using ONLY the context and prompt provided.\n"
        "NEVER make up information. NEVER mention companies or websites.\n"
        "Respond naturally and helpfully.\n"
        "Maximum 2–3 sentences."
    )

    # 3. Build judge prompt
    judge_prompt = f"""
You are an expert evaluator for conversational AI systems.
Your job is to choose which assistant reply is better.

Evaluation criteria:
- Alignment with the system prompt
- Tone matches emotion: '{emotion}'
- Urgency handling matches level: '{urgency}'
- No hallucinated assumptions
- Concise, friendly, and helpful

SYSTEM PROMPT GIVEN TO THE MODEL:
\"\"\"{system_msg}\"\"\"

USER INPUT:
\"\"\"{user_input}\"\"\"

RESPONSE A (Draft):
\"\"\"{draft_reply}\"\"\"

RESPONSE B (Revised):
\"\"\"{revised_reply}\"\"\"

Which reply is better? Answer with ONLY a single character: 'A' or 'B'.
"""

    # 4. Ask judge model
    resp = client.chat.completions.create(
        model=judge_model,
        messages=[{"role": "user", "content": judge_prompt}],
        max_tokens=1,
        temperature=0,
    )

    decision = resp.choices[0].message.content.strip().upper()

    # Fallback safety
    if decision not in ("A", "B"):
        decision = "B"

    if decision == "A":
        chosen, rejected = draft_reply, revised_reply
    else:
        chosen, rejected = revised_reply, draft_reply

    # 5. Build DPO training prompt (same format as before)
    prompt = (
        f"System: {system_msg}\n"
        f"User: {user_input}\n"
        "Assistant:"
    )

    return {
        "prompt": prompt,
        "chosen_response": chosen,
        "rejected_response": rejected,
    }


In [9]:
def load_cai_pairs_and_build_dpo_dataset_with_judge(
    cai_path: str,
    judge_model: str = "gpt-4o-mini",
) -> Dataset:
    """
    Read cai_pairs JSONL and use gpt-4o-mini to label each pair
    with chosen / rejected responses for DPO.
    """
    cai_pairs = []
    with open(cai_path, "r") as f:
        for line in f:
            if not line.strip():
                continue
            cai_pairs.append(json.loads(line))

    dpo_examples = []
    for p in tqdm(cai_pairs, desc="Labeling with judge model"):
        dpo_examples.append(make_dpo_example_with_judge(p, judge_model))

    dpo_ds = Dataset.from_list(dpo_examples)
    return dpo_ds


In [10]:
dpo_ds = load_cai_pairs_and_build_dpo_dataset_with_judge("cai_pairs.jsonl")
print(dpo_ds[0])

Labeling with judge model: 100%|██████████| 1999/1999 [15:38<00:00,  2.13it/s]

{'prompt': "System: You are a GENERAL helpful assistant, NOT a ticket agent.\nNEVER mention company names, websites, or booking services.\nThe user's detected emotion is 'happy'.\nThe urgency level is 'low'.\nANSWER THE QUESTION using ONLY the context and prompt provided.\nNEVER make up information. NEVER mention companies or websites.\nRespond naturally and helpfully.\nMaximum 2–3 sentences.\nUser: hi good morning\nAssistant:", 'chosen_response': 'Hey there! Good morning! :)', 'rejected_response': 'Hello! Welcome to AT&T Customer Service. How may I assist you today?'}


## DPO Training Process

In [11]:
base_model_name = "Qwen/Qwen2.5-1.5B-Instruct"
sft_lora_path  = "qwen2.5-cai-sft-final"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Policy model = base model + your SFT LoRA
base_policy = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)
policy_model = PeftModel.from_pretrained(
    base_policy,
    sft_lora_path,
)

# Reference model = frozen base model (no LoRA, no grad)
ref_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)
ref_model.requires_grad_(False)   # important for DPO


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotar

In [12]:
import sys
!{sys.executable} -m pip install --upgrade trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
from trl import DPOTrainer, DPOConfig

# Make sure columns match what DPOTrainer expects
print(dpo_ds.column_names)  # optional sanity check

dpo_ds = dpo_ds.rename_columns({
    "chosen_response": "chosen",
    "rejected_response": "rejected",
})

training_args = DPOConfig(
    output_dir="qwen2.5-cai-dpo",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    logging_steps=10,
    save_steps=200,
    report_to="none",
    beta=1.0,
    max_length=512,
    max_prompt_length=256,
)

# rename columns (as above) here

dpo_trainer = DPOTrainer(
    model=policy_model,
    ref_model=ref_model,
    args=training_args,
    train_dataset=dpo_ds,
    processing_class=tokenizer,
)

dpo_trainer.train()

dpo_output_dir = "qwen2.5-cai-dpo-final"
dpo_trainer.save_model(dpo_output_dir)
tokenizer.save_pretrained(dpo_output_dir)
print("DPO training finished. Saved to:", dpo_output_dir)


['prompt', 'chosen_response', 'rejected_response']


Extracting prompt in train dataset:   0%|          | 0/1999 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1999 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1999 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,10.795800
20,12.705100
30,13.686600
40,10.974000
50,11.487800
60,11.011400
70,11.602500
80,9.956000
90,13.857000
100,12.324000


DPO training finished. Saved to: qwen2.5-cai-dpo-final
